In [4]:
import os

import cv2
import numpy as np
import pandas as pd
from keras.preprocessing import image
import keras.applications
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Conv2D, Flatten, BatchNormalization
from keras.utils import print_summary
from keras import regularizers
from keras.callbacks import TensorBoard
from keras import Model
import keras.callbacks
import tensorflow

Using TensorFlow backend.


In [5]:
VAL_LABELS_FILE = "/home/jurek/Pulpit/zajecia/SUKIENKI/val/labels.txt"
TRAIN_LABELS_FILE = "/home/jurek/Pulpit/zajecia/SUKIENKI/train/labels.txt"
TEST_LABELS_FILE="/home/jurek/Pulpit/zajecia/SUKIENKI/test/labels.txt"

In [6]:
df = pd.read_csv('/home/jurek/Pulpit/zajecia/SUKIENKI/train/labels.txt', delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")


In [7]:

def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv('/home/jurek/Pulpit/zajecia/SUKIENKI/train/labels.txt', delimiter=' ', names=["path", 'beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], dtype="str")
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", class_mode="multi_output", y_col=['beige', 'black', 'blue', 'brown', 'gray', 'green', 'multicolor', 'orange', 'pink', 'red', 'violet', 'white', 'yellow', 'transaprent'], target_size=(128, 128), batch_size=20)


def show_dataset(flow):
    for img, l in flow:
        # wypisz etykietę
        print(l[0])
        # konwersja obrazu do wyświetlenia
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # wyświetl obraz
        cv2.imshow("image", img)
        # poczekaj na klawisz
        cv2.waitKey()


In [8]:
def create_model(gen):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), input_shape=(128,128, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(52, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(14, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

In [9]:
def get_callbacks():
    callbacks =[]
    mc=keras.callbacks.ModelCheckpoint("weights1.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    callbacks.append(mc)
    return callbacks

In [10]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE)
    val_gen = get_generator(VAL_LABELS_FILE)
    callbacks = get_callbacks()
    # stwórz model
    model = create_model(train_gen)
    #model.load_weights('weights1.hdf5')
    model.summary()
    # trening :)
    model.fit_generator(train_gen, epochs=30, steps_per_epoch=train_gen.n // train_gen.batch_size)
                        #validation_data = val_gen, validation_steps = val_gen.n // val_gen.batch_size) 
                        #callbacks=callbacks)

In [11]:
main()

Found 3514 validated image filenames.
Found 0 validated image filenames.








/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 3514 invalid image filename(s) in x_col="path". These filename(s) will be ignored.
  .format(n_invalid, x_col)




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 126, 126, 16)      64        
_________________________________________________________________
activation_1 (Activation)    (None, 126, 126, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 16)      2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 124, 124, 16)      64        
_________________________________________________________________
activation_2 (Activation)    (None, 124, 124, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 16)     

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 14 arrays: [array([['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
       ['0'],
   ...